In [78]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import pymysql as mdb
%matplotlib inline
import geocoder
import seaborn as sns
from bs4 import BeautifulSoup
import urllib 
import pickle

In [10]:
pools = pd.read_csv('pools.csv')

In [11]:
#a csv I complied of all the NYC rec centers with indoor pools.
pools

,swimming_pool,website_source,address
0,asser_levy,http://www.nycgovparks.org/facilities/recreati...,"392 asser levy pl, NY"
1,chelsea,http://www.nycgovparks.org/facilities/recreati...,"430 W 25th St, New York, NY 10001"
2,gertrude_erdel,http://www.nycgovparks.org/facilities/recreati...,"232 West 60th Street, ny"
3,hansborough,http://www.nycgovparks.org/facilities/recreati...,"35 west 134th street, NY"
4,tony_dapolito,http://www.nycgovparks.org/facilities/recreati...,"1 clarkson st., NY"
5,brownsville,http://www.nycgovparks.org/facilities/recreati...,"1555 linden blvd, NY"
6,metropolitan,http://www.nycgovparks.org/facilities/recreati...,"261 bedford ave, NY"
7,st.john’s,http://www.nycgovparks.org/facilities/recreati...,"1251 prospect place, NY"
8,flushing,http://www.nycgovparks.org/facilities/recreati...,"Avery Avenue and 131 Street, NY"
9,roy_wilkins,http://www.nycgovparks.org/facilities/recreati...,"17701 baisley blvd, NY"


In [17]:
#checking the full website
pools['website_source'][0]

'http://www.nycgovparks.org/facilities/recreationcenters/M164/schedule#Pool'

In [12]:
#convert addresses to latitudes and longitudes
poolist = pools['address'].values.tolist()

In [13]:
poolist

['392 asser levy pl, NY',
 '430 W 25th St, New York, NY 10001',
 '232 West 60th Street, ny',
 '35 west 134th street, NY',
 '1 clarkson st., NY',
 '1555 linden blvd, NY',
 '261 bedford ave, NY',
 '1251 prospect place, NY',
 'Avery Avenue and 131 Street, NY',
 '17701 baisley blvd, NY',
 "450 saint ann's ave, NY",
 '348 East 54th st, NY']

In [14]:
#get latitude and longitude for each pool
poolcoord = {}
for i in poolist:
    g = geocoder.google(i)
    lon=g.geojson['geometry']['coordinates'][0]
    lat=g.geojson['geometry']['coordinates'][1]
    poolcoord[i] = (lat, lon)

In [15]:
poolcoord

{'1 clarkson st., NY': (40.7296168, -74.0054061),
 '1251 prospect place, NY': (40.6738779, -73.9349038),
 '1555 linden blvd, NY': (40.65620180000001, -73.90283720000001),
 '17701 baisley blvd, NY': (40.6872438, -73.7721323),
 '232 West 60th Street, ny': (40.77142070000001, -73.98866439999999),
 '261 bedford ave, NY': (40.7150182, -73.96039019999999),
 '348 East 54th st, NY': (40.7564907, -73.9651173),
 '35 west 134th street, NY': (40.81280659999999, -73.9392448),
 '392 asser levy pl, NY': (40.7361601, -73.9756582),
 '430 W 25th St, New York, NY 10001': (40.7482487, -74.0020976),
 "450 saint ann's ave, NY": (40.8114681, -73.9140663),
 'Avery Avenue and 131 Street, NY': (40.7519669, -73.8360161)}

In [81]:
pickle.dump( poolcoord, open("coordinates_of_pools.p", "wb" ) )

In [159]:
for key in poolcoord:
    print poolcoord[key][0], poolcoord[key][1]

40.6738779 -73.9349038
40.8114681 -73.9140663
40.7296168 -74.0054061
40.7150182 -73.9603902
40.7361601 -73.9756582
40.8128066 -73.9392448
40.6872438 -73.7721323
40.7564907 -73.9651173
40.7519669 -73.8360161
40.6562018 -73.9028372
40.7714207 -73.9886644
40.7482487 -74.0020976


In [160]:
pools_coords = pd.DataFrame(poolcoord.items(), columns=['pool', 'latlon'])

In [161]:
#convert location tuple to two columns, latitude and longitude
pools_coords

,pool,latlon
0,"1251 prospect place, NY","(40.6738779, -73.9349038)"
1,"450 saint ann's ave, NY","(40.8114681, -73.9140663)"
2,"1 clarkson st., NY","(40.7296168, -74.0054061)"
3,"261 bedford ave, NY","(40.7150182, -73.9603902)"
4,"392 asser levy pl, NY","(40.7361601, -73.9756582)"
5,"35 west 134th street, NY","(40.8128066, -73.9392448)"
6,"17701 baisley blvd, NY","(40.6872438, -73.7721323)"
7,"348 East 54th st, NY","(40.7564907, -73.9651173)"
8,"Avery Avenue and 131 Street, NY","(40.7519669, -73.8360161)"
9,"1555 linden blvd, NY","(40.6562018, -73.9028372)"


In [163]:
#want separate latitude and longitude columns from the tuple
pools_coords['latlon'][0]

(40.6738779, -73.9349038)

In [165]:
from ast import literal_eval as make_tuple
make_tuple(str(pools_coords['latlon'][0]))[0]

40.6738779

In [169]:
dflocation = pools_coords['latlon']

In [170]:
latitude = []
longitude = []
for i in dflocation:
    i = str(i)
    latitude.append(make_tuple(i)[0])
    longitude.append(make_tuple(i)[1])

In [175]:
#dataframe wrangling to get the columns out

dflat = pd.DataFrame(longitude, latitude)
dflat['latitude']=dflat.index
dflat.columns=[['longitude', 'latitude']]
dflat =dflat.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
dflat.columns=[['latitude', 'longitude', 'weird']]
del dflat['weird']

In [176]:
dflat

,latitude,longitude
0,40.673878,-73.934904
1,40.811468,-73.914066
2,40.729617,-74.005406
3,40.715018,-73.960390
4,40.736160,-73.975658
5,40.812807,-73.939245
6,40.687244,-73.772132
7,40.756491,-73.965117
8,40.751967,-73.836016
9,40.656202,-73.902837


In [182]:
completepools = pd.concat([pools, dflat], axis=1, join='inner')
poolcoordinates = completepools[['address', 'latitude', 'longitude']]
poolcoordinates.to_csv('poolsandcoords')

In [432]:
completepools

,swimming_pool,website_source,address,latitude,longitude
0,asser_levy,http://www.nycgovparks.org/facilities/recreati...,"392 asser levy pl, NY",40.673878,-73.934904
1,chelsea,http://www.nycgovparks.org/facilities/recreati...,"430 W 25th St, New York, NY 10001",40.811468,-73.914066
2,gertrude_erdel,http://www.nycgovparks.org/facilities/recreati...,"232 West 60th Street, ny",40.729617,-74.005406
3,hansborough,http://www.nycgovparks.org/facilities/recreati...,"35 west 134th street, NY",40.715018,-73.960390
4,tony_dapolito,http://www.nycgovparks.org/facilities/recreati...,"1 clarkson st., NY",40.736160,-73.975658
5,brownsville,http://www.nycgovparks.org/facilities/recreati...,"1555 linden blvd, NY",40.812807,-73.939245
6,metropolitan,http://www.nycgovparks.org/facilities/recreati...,"261 bedford ave, NY",40.687244,-73.772132
7,st.john’s,http://www.nycgovparks.org/facilities/recreati...,"1251 prospect place, NY",40.756491,-73.965117
8,flushing,http://www.nycgovparks.org/facilities/recreati...,"Avery Avenue and 131 Street, NY",40.751967,-73.836016
9,roy_wilkins,http://www.nycgovparks.org/facilities/recreati...,"17701 baisley blvd, NY",40.656202,-73.902837


In [298]:
plist = completepools['swimming_pool'].tolist()
plist

['asser_levy',
 'chelsea',
 'gertrude_erdel',
 'hansborough',
 'tony_dapolito',
 'brownsville',
 'metropolitan',
 'st.john\xe2\x80\x99s',
 'flushing',
 'roy_wilkins',
 'st.mary\xe2\x80\x99s',
 'rec_center54']

In [426]:
#reading in pool schedule html for asser_levy using beautifulsoup
r = urllib.urlopen('http://www.nycgovparks.org/facilities/recreationcenters/B085/schedule#Pool').read()
soup = BeautifulSoup(r)
print type(soup)

<class 'bs4.BeautifulSoup'>


In [427]:
#exploring beautifulsoup a bit
soup.title

<title>Metropolitan Recreation Center Schedule : NYC Parks</title>

In [428]:
soup.p

<p id="site-tagline">Official Website of the New York City Department of Parks &amp; Recreation</p>

In [429]:
soup.a

<a class="no-external" href="http://www1.nyc.gov/nyc-resources/categories.page">NYC Resources</a>

In [445]:
letters[20:]

[<p class="program">\n\t\t\t\t\t\t\t\t\t3:30 p - 5:00 p\n\t\t\t\t\t\t\t\t\t<br/>\n<a class="program-popup" href="#program_9371"><strong>Children's Intro to Art</strong></a> <br/>\n\t\t\t\t\t\t\t\t\tChild\n\t\t\t\t\t\t\t\t\t<br/>\n</p>,
 <p class="program">\n\t\t\t\t\t\t\t\t\t5:00 p - 6:00 p\n\t\t\t\t\t\t\t\t\t<br/>\n<a class="program-popup" href="#program_10324"><strong>Abdominus Propulsion</strong></a> <br/>\n\t\t\t\t\t\t\t\t\tAdult, Senior\n\t\t\t\t\t\t\t\t\t<br/>\n</p>,
 <p class="program">\n\t\t\t\t\t\t\t\t\t6:30 p - 7:15 p\n\t\t\t\t\t\t\t\t\t<br/>\n<a class="program-popup" href="#program_8893"><strong>Keep In Step</strong></a> <br/>\n\t\t\t\t\t\t\t\t\tAdult, Teen\n\t\t\t\t\t\t\t\t\t<br/>\n</p>,
 <div class="center-hrs">\n<p>\n<strong>Building Hours</strong><br/>\n\t\t\t\t\t\t\t\t\t\t\t                                                                                                \t                                                     \t7:00 a - 9:30 p\n                             

In [435]:
#i want the swim programs in between the building hours, each time building hours shows up it represents the next day
other = ["program", "center-hrs"]
letters = soup.find_all(["p", "div"], other ) 

In [437]:
len(letters)

88

In [446]:
letters[0]

<div class="center-hrs">\n<p>\n<strong>Building Hours</strong><br/>\n\t\t\t\t\t\t\t\t\t\t\t                                                                                                \t                                                     \t7:00 a - 9:30 p\n                                                    \n                                                \t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t</p>\n</div>

In [123]:
test2 = letters[0]

In [125]:
print str(test2).find('swim')
#if it isn't found, it returns -1

-1


In [109]:
swimposting = []
otherposting = []
for i in letters:
    if str(i).find('Adult Lap Swim') > 0:
        swimposting.append(str(i))
    if str(i).find('Building Hours') > 0:
        swimposting.append(str(i))
    else:
        otherposting.append(str(i))

In [187]:
asserlevy = swimposting[7:]# want to replace each building hours with day of the week

In [188]:
asserlevy

['<div class="center-hrs">\n<p>\n<strong>Building Hours</strong><br/>\n\t\t\t\t\t\t\t\t\t\t\t                                                                                                \t                                                     \t6:30 a - 9:30 p\n                                                    \n                                                \t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t</p>\n</div>',
 '<p class="program">\n\t\t\t\t\t\t\t\t\t7:00 a - 9:00 a\n\t\t\t\t\t\t\t\t\t<br/>\n<a class="program-popup" href="#program_15940"><strong>Adult Lap Swim</strong></a> <br/>\n\t\t\t\t\t\t\t\t\tAdult, Senior\n\t\t\t\t\t\t\t\t\t<br/>\n</p>',
 '<p class="program">\n\t\t\t\t\t\t\t\t\t1:15 p - 3:45 p\n\t\t\t\t\t\t\t\t\t<br/>\n<a class="program-popup" href="#program_15940"><strong>Adult Lap Swim</strong></a> <br/>\n\t\t\t\t\t\t\t\t\tAdult, Senior\n\t\t\t\t\t\t\t\t\t<br/>\n</p>',
 '<p class="program">\n\t\t\t\t\t\t\t\t\t6:30 p - 9:20 p\n\t\t\t\t\t\t\t\t\t<br/>\n<a class="program-p

In [131]:
findtime = str(letters[46])

In [137]:
findtime

'<p class="program">\n\t\t\t\t\t\t\t\t\t7:00 a - 9:00 a\n\t\t\t\t\t\t\t\t\t<br/>\n<a class="program-popup" href="#program_15940"><strong>Adult Lap Swim</strong></a> <br/>\n\t\t\t\t\t\t\t\t\tAdult, Senior\n\t\t\t\t\t\t\t\t\t<br/>\n</p>'

In [135]:
#using regular expressions to pull out the time
time = re.findall('\t*[0-9]*:[0-9]*\s[a-z]\s-\s[0-9*]\s:[0-9]*\s[a-z]\n'  ,findtime)

In [144]:
time1 =  re.findall('[0-9]*:[0-9]*\s[a-z]\s-\s[0-9]*:[0-9]*\s[a-z]'  ,findtime)

In [145]:
time1

['7:00 a - 9:00 a']

In [190]:

days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [191]:
#make a loop that saves lap swim programs and substitutes the days of the week for the building hours which separate them
asser_levy_sched = {}
count = 0
for i in asserlevy:
    if str(i).find('Building Hours') > 0:
        asser_levy_sched[days_of_week[count]] = []
        count = count + 1
    if str(i).find('Adult Lap Swim') > 0:
        asser_levy_sched[days_of_week[count-1]].append(re.findall('[0-9]*:[0-9]*\s[a-z]\s-\s[0-9]*:[0-9]*\s[a-z]'  ,str(i)))

In [192]:
asser_levy_sched

{'Friday': [['7:00 a - 9:00 a'], ['1:15 p - 3:00 p'], ['7:00 p - 9:20 p']],
 'Monday': [['7:00 a - 9:00 a'], ['1:15 p - 3:45 p'], ['6:30 p - 9:20 p']],
 'Saturday': [['8:00 a - 9:00 a'], ['2:15 p - 4:30 p']],
 'Sunday': [['8:00 a - 11:00 a'], ['2:15 p - 4:30 p']],
 'Thursday': [['7:00 a - 9:00 a'], ['1:15 p - 3:30 p'], ['6:30 p - 9:20 p']],
 'Tuesday': [['7:00 a - 9:00 a'], ['1:15 p - 3:45 p'], ['6:30 p - 9:20 p']],
 'Wednesday': [['7:00 a - 9:00 a'], ['1:15 p - 3:45 p'], ['6:30 p - 9:20 p']]}

In [193]:
#want to create a dictionary for every pool let's try another, metropolitan
#reading in pool schedule html for asser_levy
r6 = urllib.urlopen(pools['website_source'][6]).read()
soup6 = BeautifulSoup(r6)
print type(soup6)


<class 'bs4.BeautifulSoup'>


In [194]:
other6 = ["program", "center-hrs"]
letters6 = soup6.find_all(["p", "div"], other6 ) 

In [196]:
swimposting6 = []
otherposting6 = []
for i in letters6:
    if str(i).find('Adult Lap Swim') > 0:
        swimposting6.append(str(i))
    if str(i).find('Building Hours') > 0:
        swimposting6.append(str(i))
    else:
        otherposting6.append(str(i))

In [199]:
metropolitan = swimposting6[7:]

In [200]:
metropolitan_sched = {}
count = 0
for i in metropolitan:
    if str(i).find('Building Hours') > 0:
        metropolitan_sched[days_of_week[count]] = []
        count = count + 1
    if str(i).find('Adult Lap Swim') > 0:
        metropolitan_sched[days_of_week[count-1]].append(re.findall('[0-9]*:[0-9]*\s[a-z]\s-\s[0-9]*:[0-9]*\s[a-z]'  ,str(i)))

In [203]:
metropolitan_sched['Friday'][0]

['7:00 a - 9:15 a']

In [204]:
metropolitan_sched

{'Friday': [['7:00 a - 9:15 a'], ['5:30 p - 9:15 p']],
 'Monday': [['7:00 a - 9:15 a'], ['1:30 p - 3:30 p'], ['6:30 p - 9:15 p']],
 'Saturday': [['7:00 a - 8:45 a'], ['4:00 p - 5:15 p']],
 'Sunday': [['10:00 a - 12:30 p']],
 'Thursday': [['7:00 a - 11:00 a'], ['1:30 p - 3:00 p'], ['5:30 p - 9:15 p']],
 'Tuesday': [['7:00 a - 11:00 a'], ['2:00 p - 3:00 p'], ['5:30 p - 9:15 p']],
 'Wednesday': [['7:00 a - 9:15 a'], ['1:30 p - 3:30 p'], ['6:30 p - 9:15 p']]}

In [208]:

df = pd.DataFrame.from_dict(metropolitan_sched, orient='index')

In [212]:
df = df.transpose()

In [216]:
del df['Metropolitan Pool'] 

In [219]:
df['Pool'] = 'Metropolitan Pool'

In [220]:
df

,Monday,Tuesday,Friday,Wednesday,Thursday,Sunday,Saturday,Pool
0,[7:00 a - 9:15 a],[7:00 a - 11:00 a],[7:00 a - 9:15 a],[7:00 a - 9:15 a],[7:00 a - 11:00 a],[10:00 a - 12:30 p],[7:00 a - 8:45 a],Metropolitan Pool
1,[1:30 p - 3:30 p],[2:00 p - 3:00 p],[5:30 p - 9:15 p],[1:30 p - 3:30 p],[1:30 p - 3:00 p],None,[4:00 p - 5:15 p],Metropolitan Pool
2,[6:30 p - 9:15 p],[5:30 p - 9:15 p],None,[6:30 p - 9:15 p],[5:30 p - 9:15 p],None,None,Metropolitan Pool


In [227]:
#pipeline to scrape each pool, collect times for days of the week, convert to a dataframe then concatenate with the other dataframes
#this doesn't work due to inconsistencies on the different pool websites, some say lap swim, others adult lap swim and others adult swim.
#there are other non-lap swim programs so i have to scrape each separately and check.
dflist = ['df1','df2','df3', 'df4','df5', 'df6', 'df7', 'df8', 'df9', 'df10', 'df11', 'df12']
poolist = ['asser_levy', 'chelsea', 'gertrude_erdel','hansborough','tony_dapolito','brownsville','metropolitan','st.johns','flushing','roy_wilkins','st.marys','rec_center54']
failist = []

for j in range(4, 13):
    try:
        r = urllib.urlopen(pools['website_source'][j]).read()
        soup = BeautifulSoup(r)
        letters  = soup.find_all(["p", "div"], ["program", "center-hrs"] ) 
        swimposting = []
        otherposting = []
        for i in letters:
            if str(i).find('Lap Swim') > 0:
                swimposting.append(str(i))
            if str(i).find('Building Hours') > 0:
                swimposting.append(str(i))
            else:
                otherposting.append(str(i))
        pools['swimming_pool'][j] = swimposting[7:-7]
        sched = {}
        count = 0
        for s in pools['swimming_pool'][j]:
            if str(s).find('Building Hours') > 0:
                sched[days_of_week[count]] = []
                count = count + 1
            if str(s).find('Lap Swim') > 0:
                sched[days_of_week[count-1]].append(re.findall('[0-9]*:[0-9]*\s[a-z]\s-\s[0-9]*:[0-9]*\s[a-z]'  ,str(s)))
        dflist[j] = pd.DataFrame.from_dict(sched, orient='index')
        dflist[j] = dflist[j].transpose()
        eeps = poolist[j]
        dflist[j]['Pool'] = eeps
    except:
        failist.append(j)

SyntaxError: can't assign to operator (<ipython-input-227-3a7b8bfda95c>, line 3)

In [224]:
for i in range(0, 12):
    print pools['swimming_pool'][i]

asser_levy
chelsea
gertrude_erdel
hansborough
tony_dapolito
brownsville
metropolitan
st.john’s
flushing
roy_wilkins
st.mary’s
rec_center54


In [233]:
r = urllib.urlopen(pools['website_source'][0]).read()
soup = BeautifulSoup(r)
letters  = soup.find_all(["p", "div"], ["program", "center-hrs"] ) 
swimposting = []
otherposting = []
for i in letters:
    if str(i).find('Adult Lap Swim') > 0:
        swimposting.append(str(i))
    if str(i).find('Building Hours') > 0:
        swimposting.append(str(i))
    else:
        otherposting.append(str(i))


In [236]:
pools['swimming_pool'][0] = swimposting[7:]
sched = {}
count = 0
for i in pools['swimming_pool'][0]:
    if str(i).find('Building Hours') > 0:
        sched[days_of_week[count]] = []
        count = count + 1
    if str(i).find('Adult Lap Swim') > 0:
        sched[days_of_week[count-1]].append(re.findall('[0-9]*:[0-9]*\s[a-z]\s-\s[0-9]*:[0-9]*\s[a-z]'  ,str(i)))


In [237]:
sched

{'Friday': [['7:00 a - 9:00 a'], ['1:15 p - 3:00 p'], ['7:00 p - 9:20 p']],
 'Monday': [['7:00 a - 9:00 a'], ['1:15 p - 3:45 p'], ['6:30 p - 9:20 p']],
 'Saturday': [['8:00 a - 9:00 a'], ['2:15 p - 4:30 p']],
 'Sunday': [['8:00 a - 11:00 a'], ['2:15 p - 4:30 p']],
 'Thursday': [['7:00 a - 9:00 a'], ['1:15 p - 3:30 p'], ['6:30 p - 9:20 p']],
 'Tuesday': [['7:00 a - 9:00 a'], ['1:15 p - 3:45 p'], ['6:30 p - 9:20 p']],
 'Wednesday': [['7:00 a - 9:00 a'], ['1:15 p - 3:45 p'], ['6:30 p - 9:20 p']]}

In [238]:

#dataframe wrangling to get it in the format i want, i'll then join all dataframes from the separate pools
df0 = pd.DataFrame.from_dict(sched, orient='index')
#df0 = df0.transpose()
#df0['Pool'] = pools['swimming_pool'][0]

In [246]:
df0 = df0.transpose()

In [247]:
del df0['Pool'] 

In [249]:
df0 = df0.transpose()

In [251]:
df0['Pool'] = 'asser levy'

In [252]:
df0

,Monday,Tuesday,Friday,Wednesday,Thursday,Sunday,Saturday,Pool
0,[7:00 a - 9:00 a],[7:00 a - 9:00 a],[7:00 a - 9:00 a],[7:00 a - 9:00 a],[7:00 a - 9:00 a],[8:00 a - 11:00 a],[8:00 a - 9:00 a],asser levy
1,[1:15 p - 3:45 p],[1:15 p - 3:45 p],[1:15 p - 3:00 p],[1:15 p - 3:45 p],[1:15 p - 3:30 p],[2:15 p - 4:30 p],[2:15 p - 4:30 p],asser levy
2,[6:30 p - 9:20 p],[6:30 p - 9:20 p],[7:00 p - 9:20 p],[6:30 p - 9:20 p],[6:30 p - 9:20 p],None,None,asser levy


In [259]:
r = urllib.urlopen(pools['website_source'][1]).read()
soup = BeautifulSoup(r)
letters  = soup.find_all(["p", "div"], ["program", "center-hrs"] ) 
swimposting = []
otherposting = []
for i in letters:
    if str(i).find('Lap Swim') > 0:
        swimposting.append(str(i))
    if str(i).find('Building Hours') > 0:
        swimposting.append(str(i))
    else:
        otherposting.append(str(i))


In [270]:
swimposting[7:-6]

['<div class="center-hrs">\n<p>\n<strong>Building Hours</strong><br/>\n\t\t\t\t\t\t\t\t\t\t\t                                                                                                \t                                                     \t6:30 a - 10:00 p\n                                                    \n                                                \t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t</p>\n</div>',
 '<p class="program">\n\t\t\t\t\t\t\t\t\t7:00 a - 10:00 a\n\t\t\t\t\t\t\t\t\t<br/>\n<a class="program-popup" href="#program_14800"><strong>Lap Swim</strong></a> <br/>\n\t\t\t\t\t\t\t\t\tAdult, Senior\n\t\t\t\t\t\t\t\t\t<br/>\n</p>',
 '<p class="program">\n\t\t\t\t\t\t\t\t\t2:00 p - 3:45 p\n\t\t\t\t\t\t\t\t\t<br/>\n<a class="program-popup" href="#program_14800"><strong>Lap Swim</strong></a> <br/>\n\t\t\t\t\t\t\t\t\tAdult, Senior\n\t\t\t\t\t\t\t\t\t<br/>\n</p>',
 '<p class="program">\n\t\t\t\t\t\t\t\t\t9:00 p - 10:00 p\n\t\t\t\t\t\t\t\t\t<br/>\n<a class="program-popup" hre

In [267]:
pools['swimming_pool'][1] = swimposting[7:-7]
sched = {}
count = 0
for i in pools['swimming_pool'][1]:
    if str(i).find('Building Hours') > 0:
        sched[days_of_week[count]] = []
        count = count + 1
    if str(i).find('Lap Swim') > 0:
        sched[days_of_week[count-1]].append(re.findall('[0-9]*:[0-9]*\s[a-z]\s-\s[0-9]*:[0-9]*\s[a-z]'  ,str(i)))


In [268]:
df1 = pd.DataFrame.from_dict(sched, orient='index')
df1 = df1.transpose()
df1['Pool'] = 'chelsea'

In [269]:
df1

,Monday,Tuesday,Friday,Wednesday,Thursday,Sunday,Saturday,Pool
0,[7:00 a - 10:00 a],[7:00 a - 10:00 a],[7:00 a - 9:30 a],[7:00 a - 9:30 a],[7:00 a - 10:00 a],[8:00 a - 11:00 a],[7:00 a - 8:00 a],chelsea
1,[2:00 p - 3:45 p],[1:00 p - 3:45 p],[2:00 p - 3:45 p],[2:00 p - 3:45 p],[1:00 p - 3:45 p],[3:00 p - 4:00 p],None,chelsea
2,[9:00 p - 10:00 p],[6:00 p - 7:00 p],[9:00 p - 10:00 p],[9:00 p - 10:00 p],[6:00 p - 7:00 p],None,None,chelsea
3,None,None,None,None,[9:00 p - 10:00 p],None,None,chelsea


In [271]:
r = urllib.urlopen(pools['website_source'][2]).read()
soup = BeautifulSoup(r)
letters  = soup.find_all(["p", "div"], ["program", "center-hrs"] ) 
swimposting = []
otherposting = []
for i in letters:
    if str(i).find('Lap Swim') > 0:
        swimposting.append(str(i))
    if str(i).find('Building Hours') > 0:
        swimposting.append(str(i))
    else:
        otherposting.append(str(i))

In [274]:
swimposting[7:-7]

['<div class="center-hrs">\n<p>\n<strong>Building Hours</strong><br/>\n\t\t\t\t\t\t\t\t\t\t\t                                                                                                \t                                                     \t6:30 a - 9:30 p\n                                                    \n                                                \t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t</p>\n</div>',
 '<p class="program">\n\t\t\t\t\t\t\t\t\t7:00 a - 9:45 a\n\t\t\t\t\t\t\t\t\t<br/>\n<a class="program-popup" href="#program_16641"><strong>Lap Swim</strong></a> <br/>\n\t\t\t\t\t\t\t\t\tAdult, Senior\n\t\t\t\t\t\t\t\t\t<br/>\n</p>',
 '<p class="program">\n\t\t\t\t\t\t\t\t\t6:15 p - 9:00 p\n\t\t\t\t\t\t\t\t\t<br/>\n<a class="program-popup" href="#program_16641"><strong>Lap Swim</strong></a> <br/>\n\t\t\t\t\t\t\t\t\tAdult, Senior\n\t\t\t\t\t\t\t\t\t<br/>\n</p>',
 '<div class="center-hrs">\n<p>\n<strong>Building Hours</strong><br/>\n\t\t\t\t\t\t\t\t\t\t\t                     

In [276]:
pools['swimming_pool'][2] = swimposting[7:-7]
sched = {}
count = 0
for i in pools['swimming_pool'][2]:
    if str(i).find('Building Hours') > 0:
        sched[days_of_week[count]] = []
        count = count + 1
    if str(i).find('Lap Swim') > 0:
        sched[days_of_week[count-1]].append(re.findall('[0-9]*:[0-9]*\s[a-z]\s-\s[0-9]*:[0-9]*\s[a-z]'  ,str(i)))

        

In [281]:
df2 = pd.DataFrame.from_dict(sched, orient='index')
df2 = df2.transpose()
df2['Pool'] = 'Gertrude Eredel'

In [282]:
df2

,Monday,Tuesday,Friday,Wednesday,Thursday,Sunday,Saturday,Pool
0,[7:00 a - 9:45 a],[7:00 a - 9:45 a],[7:00 a - 9:45 a],[7:00 a - 9:45 a],[7:00 a - 9:45 a],None,[8:00 a - 9:45 a],Gertrude Eredel
1,[6:15 p - 9:00 p],[12:00 p - 2:00 p],[1:00 p - 3:45 p],[6:15 p - 9:00 p],[12:00 p - 2:00 p],None,None,Gertrude Eredel
2,None,[6:15 p - 9:00 p],[7:15 p - 9:00 p],None,[6:15 p - 9:00 p],None,None,Gertrude Eredel


In [283]:
r = urllib.urlopen(pools['website_source'][3]).read()
soup = BeautifulSoup(r)
letters  = soup.find_all(["p", "div"], ["program", "center-hrs"] ) 
swimposting = []
otherposting = []
for i in letters:
    if str(i).find('Lap Swim') > 0:
        swimposting.append(str(i))
    if str(i).find('Building Hours') > 0:
        swimposting.append(str(i))
    else:
        otherposting.append(str(i))

In [285]:
swimposting[7:-7]


['<div class="center-hrs">\n<p>\n<strong>Building Hours</strong><br/>\n\t\t\t\t\t\t\t\t\t\t\t                                                                                                \t                                                     \t6:30 a - 9:15 p\n                                                    \n                                                \t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t</p>\n</div>',
 '<p class="program">\n\t\t\t\t\t\t\t\t\t7:00 a - 8:45 a\n\t\t\t\t\t\t\t\t\t<br/>\n<a class="program-popup" href="#program_17138"><strong>Adult Lap Swim</strong></a> <br/>\n\t\t\t\t\t\t\t\t\tAdult, Senior\n\t\t\t\t\t\t\t\t\t<br/>\n</p>',
 '<p class="program">\n\t\t\t\t\t\t\t\t\t2:00 p - 3:00 p\n\t\t\t\t\t\t\t\t\t<br/>\n<a class="program-popup" href="#program_17138"><strong>Adult Lap Swim</strong></a> <br/>\n\t\t\t\t\t\t\t\t\tAdult, Senior\n\t\t\t\t\t\t\t\t\t<br/>\n</p>',
 '<p class="program">\n\t\t\t\t\t\t\t\t\t7:15 p - 9:00 p\n\t\t\t\t\t\t\t\t\t<br/>\n<a class="program-p

In [286]:
pools['swimming_pool'][3] = swimposting[7:-7]
sched = {}
count = 0
for i in pools['swimming_pool'][3]:
    if str(i).find('Building Hours') > 0:
        sched[days_of_week[count]] = []
        count = count + 1
    if str(i).find('Lap Swim') > 0:
        sched[days_of_week[count-1]].append(re.findall('[0-9]*:[0-9]*\s[a-z]\s-\s[0-9]*:[0-9]*\s[a-z]'  ,str(i)))

        

In [287]:
df3 = pd.DataFrame.from_dict(sched, orient='index')
df3 = df3.transpose()
df3['Pool'] = 'Hansborough'

In [414]:
cols = df3.columns.tolist()


In [415]:
cols = cols[-1:] + cols[:-1]

In [416]:
df3 = df3[cols]

In [418]:
cols = ['Pool', 'Monday','Tuesday', 'Wednesday','Thursday','Friday','Saturday','Sunday']

In [419]:
df3 = df3[cols]

In [420]:
df3

,Pool,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,Hansborough,[7:00 a - 8:45 a],[9:00 a - 10:00 a],[7:00 a - 8:45 a],[9:00 a - 10:00 a],[7:00 a - 8:45 a],[3:00 p - 4:00 p],None
1,Hansborough,[2:00 p - 3:00 p],[1:15 p - 3:45 p],[2:00 p - 3:00 p],[1:15 p - 3:45 p],[3:00 p - 4:15 p],None,None
2,Hansborough,[7:15 p - 9:00 p],[7:15 p - 9:00 p],[7:15 p - 9:00 p],[7:15 p - 9:00 p],[7:15 p - 9:00 p],None,None


In [397]:
r = urllib.urlopen(pools['website_source'][11]).read()
soup = BeautifulSoup(r)
letters  = soup.find_all(["p", "div"], ["program", "center-hrs"] ) 
swimposting = []
otherposting = []
for i in letters:
    if str(i).find('Adult Lap Swim') > 0:
        swimposting.append(str(i))
    if str(i).find('Building Hours') > 0:
        swimposting.append(str(i))
    else:
        otherposting.append(str(i))
 

In [399]:
swimposting[7:-7]


['<div class="center-hrs">\n<p>\n<strong>Building Hours</strong><br/>\n\t\t\t\t\t\t\t\t\t\t\t                                                                                                \t                                                     \t6:30 a - 9:30 p\n                                                    \n                                                \t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t</p>\n</div>',
 '<p class="program">\n\t\t\t\t\t\t\t\t\t7:00 a - 9:45 a\n\t\t\t\t\t\t\t\t\t<br/>\n<a class="program-popup" href="#program_16669"><strong>Adult Lap Swim</strong></a> <br/>\n\t\t\t\t\t\t\t\t\tAdult, Senior\n\t\t\t\t\t\t\t\t\t<br/>\n</p>',
 '<p class="program">\n\t\t\t\t\t\t\t\t\t12:15 p - 3:00 p\n\t\t\t\t\t\t\t\t\t<br/>\n<a class="program-popup" href="#program_16669"><strong>Adult Lap Swim</strong></a> <br/>\n\t\t\t\t\t\t\t\t\tAdult, Senior\n\t\t\t\t\t\t\t\t\t<br/>\n</p>',
 '<p class="program">\n\t\t\t\t\t\t\t\t\t7:15 p - 9:00 p\n\t\t\t\t\t\t\t\t\t<br/>\n<a class="program-

In [400]:
sched = {}
count = 0
for i in swimposting[7:-7]:
    if str(i).find('Building Hours') > 0:
        sched[days_of_week[count]] = []
        count = count + 1
    if str(i).find('Adult Lap Swim') > 0:
        sched[days_of_week[count-1]].append(re.findall('[0-9]*:[0-9]*\s[a-z]\s-\s[0-9]*:[0-9]*\s[a-z]'  ,str(i)))
df11 = pd.DataFrame.from_dict(sched, orient='index')
df11 = df11.transpose()
df11['Pool'] = 'Recreation Center 54'

In [402]:
allpools = pd.concat([df0,df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11])

In [404]:
allpools
allpools.to_csv('nycrecreationcenterlaphours.csv')

In [406]:
allpools
allpools = allpools.reset_index()

In [408]:
del allpools['index']


In [413]:
allpools['Monday'][0][0]


'7:00 a - 9:00 a'

In [421]:
allpools = allpools[cols]

In [422]:
allpools

,Pool,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,asser levy,[7:00 a - 9:00 a],[7:00 a - 9:00 a],[7:00 a - 9:00 a],[7:00 a - 9:00 a],[7:00 a - 9:00 a],[8:00 a - 9:00 a],[8:00 a - 11:00 a]
1,asser levy,[1:15 p - 3:45 p],[1:15 p - 3:45 p],[1:15 p - 3:45 p],[1:15 p - 3:30 p],[1:15 p - 3:00 p],[2:15 p - 4:30 p],[2:15 p - 4:30 p]
2,asser levy,[6:30 p - 9:20 p],[6:30 p - 9:20 p],[6:30 p - 9:20 p],[6:30 p - 9:20 p],[7:00 p - 9:20 p],None,None
3,chelsea,[7:00 a - 10:00 a],[7:00 a - 10:00 a],[7:00 a - 9:30 a],[7:00 a - 10:00 a],[7:00 a - 9:30 a],[7:00 a - 8:00 a],[8:00 a - 11:00 a]
4,chelsea,[2:00 p - 3:45 p],[1:00 p - 3:45 p],[2:00 p - 3:45 p],[1:00 p - 3:45 p],[2:00 p - 3:45 p],None,[3:00 p - 4:00 p]
5,chelsea,[9:00 p - 10:00 p],[6:00 p - 7:00 p],[9:00 p - 10:00 p],[6:00 p - 7:00 p],[9:00 p - 10:00 p],None,None
6,chelsea,None,None,None,[9:00 p - 10:00 p],None,None,None
7,Gertrude Eredel,[7:00 a - 9:45 a],[7:00 a - 9:45 a],[7:00 a - 9:45 a],[7:00 a - 9:45 a],[7:00 a - 9:45 a],[8:00 a - 9:45 a],None
8,Gertrude Eredel,[6:15 p - 9:00 p],[12:00 p - 2:00 p],[6:15 p - 9:00 p],[12:00 p - 2:00 p],[1:00 p - 3:45 p],None,None
9,Gertrude Eredel,None,[6:15 p - 9:00 p],None,[6:15 p - 9:00 p],[7:15 p - 9:00 p],None,None


In [153]:
address = '799 west 187th street, ny'
g = geocoder.google(address)
lon=g.geojson['geometry']['coordinates'][0]
lat=g.geojson['geometry']['coordinates'][1]
lathigh = float(lat + 0.0025)
latlow = float(lat - 0.0025)
lonhigh = float(lon + 0.0025)
lonlow = float(lon - 0.0025)
latlonhigh = str(lathigh), str(lonhigh)
latlonlow = str(latlow), str(lonlow)
lathighlonlow = str(lathigh), str(lonlow)
latlowlonhigh = str(latlow), str(lonhigh)

try:
    g =geocoder.google(latlonhigh, method='reverse')
    zipcodea = g.address.split('NY ')[1].split(',')[0]
except:
    print 'hi'
try:
    g =geocoder.google(latlonlow, method='reverse')
    zipcodeb = g.address.split('NY ')[1].split(',')[0]
except:
    print 'hi'
g =geocoder.google(lathighlonlow, method='reverse')
try:
    zipcodec = g.address.split('NY ')[1].split(',')[0]
except:
    print 'hi'
g =geocoder.google(latlowlonhigh, method='reverse')
try:
    zipcoded = g.address.split('NY ')[1].split(',')[0]
except:
    print 'hi'

hi


In [157]:
zipcodec


NameError: name 'zipcodec' is not defined

In [147]:
latlonlow

('40.8524484', '-73.9384773')

In [148]:
lathighlonlow

('40.8574484', '-73.9384773')

In [151]:
for i in range(0,4):
    print i

0
1
2
3
